# Testing AI index document frequencies in queries (made necessary code changes in rank_doc and rank_doc_tfidf)

## To test it out, you have to manually alter these functions

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re,os
import nltk
import pickle
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
import numpy as np
from operator import itemgetter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
os.getcwd()

'C:\\Users\\User\\Desktop\\479-A3'

In [10]:
os.listdir()

['.ipynb_checkpoints',
 'A3-Q1-2-3-final.ipynb',
 'ai_index_df',
 'desktop.ini',
 'documents-ai',
 'documents-ai-2',
 'documents-ai-3',
 'documents-ai-filtered',
 'documents-final',
 'Filtering-duplicates.ipynb',
 'final_index_concordia',
 'Preparing-documents.ipynb',
 'Q4-AI-index.ipynb',
 'Q5.ipynb',
 'term_freq_concordia',
 'urls_run_final.csv']

In [11]:
#Merged_block = open("Final_index")
Merged_block = pickle.load(open("final_index_concordia",mode='rb'))

#Term frequency
term_freq = pickle.load(open("term_freq_concordia",mode="rb"))

#ai index
ai_index = pickle.load(open("ai_index_df",mode="rb"))

In [13]:
ai_index.keys()

dict_keys(['artificial', 'intelligence', 'machine', 'learning', 'ai', 'neural', 'networks', 'cognitive', 'supervised', 'unsupervised', 'semantic', 'analysis', 'chatbot', 'science', 'algorithm', 'data', 'mining', 'big', 'turing', 'analytics', 'cluster', 'engineering', 'reinforcement', 'nlp', 'deep', 'classification', 'regression', 'probability', 'gradient', 'machines'])

In [51]:
"Another approach to get length of a document.. Counts the number of times a token occured in some document(its length)."
def get_doc_length(doc_id):
    
    os.chdir("documents-final")
    f = open(str(doc_id)+".txt","r",encoding="utf=8")
    text= f.read()
    os.chdir("..")
    
    return len(text.split(" "))

"Calculates the average length of all documents using the get_doc_length method."
def get_collection_length():
    
    total = 0
    
    os.chdir("documents-final")
    docs = os.listdir()
    os.chdir("..")
    
    for doc in docs:
        if doc.split(".")[0].isdigit():
            total+=get_doc_length(doc)
    
    
        
    return total/40211
        

"BM25 Ranking formula implementation"
def rank_doc(doc_id,term):
        
    N = 17393 # Total documents number
    k1 = 0.5 # tuning parameter
    b = 0.5 # tuning parameter
    postings_list = []
    
    avg_doc_length = 200.04749944045162


    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    if term in ai_index.keys():
        document_frequency = ai_index[term]
    else:
        document_frequency = len(Merged_block[term])
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    numerator = (k1+1)*term_frequency
    denomenator = k1*((1-b)+ (b*(doc_length/avg_doc_length))) + term_frequency
    
    ratio = numerator/denomenator
    
    weight = idf * ratio
    
    return weight

def rank_doc_tfidf(doc_id,term):
    
    N = 17393 # Total documents number
    postings_list = []
    
    avg_doc_length = 200.04749944045162

    doc_length = get_doc_length(doc_id)
        
    
    "Find term frequency"
    
    term_frequency = term_freq[term]
    
    "Find document frequency"
    if term in ai_index.keys():
        document_frequency = ai_index[term]
    else:
        document_frequency = len(Merged_block[term])
            
    
     
    if document_frequency == 0:
        #"term doesn't exist in document"
        return 0
    
    "Formula steps:"
    
    idf = np.log(N/document_frequency)
    tf = 1 + np.log(term_frequency)
    weight = idf*tf
    
    
    return weight


In [44]:
"This function can run a single query, multi-term AND query, and a multi-term AND query with 1 OR."

def retrieve_docs(query,OR=False,vector_norm=False,tfidf=False,num_ret=10):
    
    terms = query.split(" ")
    temp_postings = []
    postings_list = []
    "Run the following if its a single term query"
    
    if len(terms) == 1:
            
        if terms[0] in Merged_block.keys():
            documents = set(Merged_block[terms[0]])
        else:
            print("Term doesn't exist in the IR system.")
            return
        
        "To store ranked documents"
        ranked_dict = {}
        
        "Rank documents and store document-rankingweight mappings"
        
        for document in documents:
            if tfidf == False:
                ranked_dict[document] = rank_doc(document,terms[0])
            else:
                ranked_dict[document] = rank_doc_tfidf(document,terms[0])


        
        "Sort ranked results"
        ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
        
        return ranked_dict
        
    "Retrieve the postings of that term"
    for term in terms:
        if (term) in Merged_block.keys():
            postings_list = list(set(Merged_block[(term)]))
                
           
        if len(postings_list) == 0:
            print("Term not found in IR system, and thus will be ignored.")
            postings_list = []
            continue
        temp_postings.append(postings_list)    
        
        
    "Optimizes querying by dealing with the shortest postings lists first"    
    temp_postings.sort(key=len)
    
    documents = set()
    
    "AND + OR logic"
    for i in range(len(temp_postings)):
        if i == 0:
            documents = documents.union(set(temp_postings[i]))
        if i == 1 and OR == True:
            documents = documents.union(set(temp_postings[i]))
        else:
            documents = documents.intersection(set(temp_postings[i]))
     
    "Store document-rankingWeight mappings"
    ranked_dict = {}
    "Accumaltor for all the query terms in a doc as per the BM25 formula"
    total_rank = 0
    
    for document in documents:
        for term in terms:
            if tfidf == False:
                total_rank+= rank_doc(document,term)
            else:
                total_rank+= rank_doc_tfidf(document,term)


        ranked_dict[document] = total_rank
        
    "Sort ranked results"       
    ranked_dict = sorted(ranked_dict.items(), key=itemgetter(1),reverse=True)
    
    #return list(temp_dict.keys())[:10]
    return ranked_dict[:num_ret]



In [45]:
query1 = "artificial intelligence"

In [48]:
# Old df weights
retrieve_docs((query1),tfidf=False,num_ret=10)

[(17402, 2897.974674452672),
 (22513, 2886.733611141023),
 (11249, 2875.473195625473),
 (14832, 2864.212962234033),
 (19951, 2852.9492674494963),
 (20461, 2841.73772648784),
 (29676, 2830.475833519786),
 (21484, 2819.2242945687285),
 (8678, 2807.9841587674196),
 (12261, 2796.750007675136)]

In [52]:
# Ai index df weights
retrieve_docs((query1),tfidf=False,num_ret=10)

[(17402, 1215.959398262353),
 (22513, 1211.2427910878355),
 (11249, 1206.5181237647414),
 (14832, 1201.7935322950038),
 (19951, 1197.067499183405),
 (20461, 1192.3631883795106),
 (29676, 1187.6379057089262),
 (21484, 1182.91693543015),
 (8678, 1178.2007145665102),
 (12261, 1173.4869863632612)]